<a href="https://colab.research.google.com/github/anikch/paddy/blob/main/paddy_doctor_5_augmentation%2Befficientnet_b4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pytorch_lightning==1.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 582 kB 16.8 MB/s 
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.7.2
    Uninstalling pytorch-lightning-1.7.2:
      Successfully uninstalled pytorch-lightning-1.7.2


In [3]:
!pip install "opencv-python-headless<4.3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch.multiprocessing as mp
import torch
import sys
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler, Adam, SGD
import torch.nn.functional as F
from torch.utils.data.sampler import WeightedRandomSampler
import torchvision.models as models
from collections import OrderedDict
from torch.autograd import Variable
import numpy as np
import pandas as pd
import time
from typing import Optional # required for "Optional[type]"
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# lightning related imports
import pytorch_lightning as pl
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.strategies.ddp import DDPStrategy
from torchmetrics import Accuracy

import albumentations as alb

import warnings
warnings.filterwarnings("ignore", message=".*does not have many workers which may be a bottleneck.*")

import os
ImageFile.LOAD_TRUNCATED_IMAGES = True

import posixpath
#import onnx
#from onnx_tf.backend import prepare

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:

colab= '/content/drive/MyDrive'
local= r'G:\My Drive'

code_runtime= colab

file1= 'thesis/dataset/paddy_doctor/train.csv'
inp_file = posixpath.join(code_runtime, file1)

train_table=pd.read_csv(inp_file)
train_table

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45
...,...,...,...,...
10402,107607.jpg,tungro,Zonal,55
10403,107811.jpg,tungro,Zonal,55
10404,108547.jpg,tungro,Zonal,55
10405,110245.jpg,tungro,Zonal,55


In [7]:
class ImageLabelDataset(Dataset):
    def __init__(self, image_id, path, y_true, isTrain=0):
        super().__init__()
        self.image_id = image_id
        self.path = path
        self.y_true = y_true
        self.isTrain = isTrain

    def __len__(self):
        return len(self.image_id)
    
    @property
    def train_labels(self):
        # warnings.warn("train_labels has been renamed targets")
        return self.y_true
    
    def __getitem__(self, index):
        label = self.y_true[index]
        label_tensor = int(label)
        # label_tensor = torch.tensor(label, dtype=torch.float32)
        tensorimage = self.preprocess_image(self.image_id[index], self.path[index])
        return tensorimage, label_tensor

    def preprocess_image(self, image_id, path):        
        
        image_transform_train = alb.Compose([
            alb.Resize(p=1.0, width=380, height=380, interpolation=3),
            # alb.ToGray(p=1.0),
#             alb.RandomResizedCrop(p=1.0, width=380, height=380, interpolation=3),
            alb.Transpose(p=0.5),
            alb.HorizontalFlip(p=0.5),
            alb.VerticalFlip(p=0.5),
            alb.ShiftScaleRotate(p=0.5),
            alb.CLAHE(p=0.5, clip_limit=(1, 4), tile_grid_size=(12, 12)),
            alb.GridDistortion(p=0.5, num_steps=5, distort_limit=(-0.4, 0.4), interpolation=0, border_mode=2),
            # alb.HueSaturationValue(
            #     hue_shift_limit=0.2, 
            #     sat_shift_limit=0.2, 
            #     val_shift_limit=0.2, 
            #     p=0.5
            # ),
            # alb.RandomBrightnessContrast(
            #     brightness_limit=(-0.1,0.1), 
            #     contrast_limit=(-0.1, 0.1), 
            #     p=0.5
            # ),
            alb.GridDropout(p=0.6, ratio=0.4, holes_number_x=3, holes_number_y=3, random_offset=True)
        ], p=1.0)
        
        image_transform_test = alb.Compose([
            alb.Resize(p=1.0, width=380, height=380, interpolation=3),
            # alb.ToGray(p=1.0),
        ], p=1.0)
        
        preprocess = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        
       # folder = '/content/drive/MyDrive/thesis/dataset/paddy_doctor/train_images/{}'.format(path)
       # if(self.isTrain == 2):
       #     folder = '/content/drive/MyDrive/thesis/dataset/paddy_doctor/test_images
        
        image = Image.open(path)
        
        if(self.isTrain < 1):
            transform_image = image_transform_train(image=np.array(image))['image']
            return preprocess(transform_image)
        else:
            transform_image = image_transform_test(image=np.array(image))['image']
            return preprocess(transform_image)
    
    def get_dummies(self, attribute_id):
        label_tensor = torch.zeros((1, 102))
        label_tensor[0, int(attribute_id)-1] = 1
        return label_tensor

In [8]:
def freeze_All(module):
    if hasattr(module, 'weight'):
        module.weight.requires_grad_(False)
    if hasattr(module, 'bias'):
        # print(module)
        # module.bias.requires_grad_(False)
        pass
    module.eval()

In [9]:
def create_model(output=2):
    model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b4', pretrained=True)
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            freeze_All(module)
    
    for module in model.stem.modules():
        if isinstance(module, nn.Conv2d):
            freeze_All(module)
    
    for module in model.layers[:3].modules():
        if isinstance(module, nn.Conv2d):
            freeze_All(module)

    num_ftrs = model.classifier.fc.in_features
#     model.classifier.fc = nn.Linear(num_ftrs, Output_Class)

    model.classifier.fc = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(num_ftrs, output)
    )
    
    # model.load_state_dict(torch.load('/content/drive/MyDrive/Data/Inception_model_softmax.pt'))    
    
    return model

In [13]:
labels = {
    'normal': 0, 'bacterial_leaf_blight': 1, 'bacterial_leaf_streak': 2,
    'bacterial_panicle_blight': 3, 'blast': 4, 'brown_spot': 5, 'dead_heart': 6,
    'downy_mildew': 7, 'hispa': 8, 'tungro': 9,
}
rev_labels = {
    0: 'normal', 1: 'bacterial_leaf_blight', 2: 'bacterial_leaf_streak',
    3: 'bacterial_panicle_blight', 4: 'blast', 5: 'brown_spot', 6: 'dead_heart',
    7: 'downy_mildew', 8: 'hispa', 9: 'tungro',
}
class PaddyNeuralNet(pl.LightningModule):
    def __init__(self, learning_rate=0.1, batch_size=8, output=2):
        super().__init__()
        # Boring model
        self.accuracy = Accuracy()
        self.model = create_model(output)
        
        self.loss_fn = nn.CrossEntropyLoss()
        
        # Hyperparameters
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        # Log hyperparameters
        self.save_hyperparameters()

    def forward(self, X_batch):
        return self.model(X_batch)

    def training_step(self, batch, batch_idx):
        X_batch, Y_batch = batch
        logits = self.forward(X_batch)
        
        loss_val = self.loss_fn(logits.squeeze(), Y_batch)

        acc = self.accuracy(logits, Y_batch)
        self.log('train_loss', loss_val, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)

        return loss_val
    
    def training_epoch_end(self, outputs):
        # print("Train end ep")
        # torch.save(self.model.state_dict(), "models/temp_eff_model_softmax.pt")
        pass
            
    def validation_step(self, batch, batch_idx):
        X_batch, Y_batch = batch
        logits = self.forward(X_batch)

        loss_val = self.loss_fn(logits.squeeze(), Y_batch)
        
        acc = self.accuracy(logits, Y_batch)
        self.log('val_loss', loss_val, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss_val

    def test_step(self, batch, batch_idx):
        X_batch, Y_batch = batch
        logits = self.forward(X_batch)
        
        #.unsqueeze(1)
        loss_val = self.loss_fn(logits.squeeze(), Y_batch)
        
        acc = self.accuracy(logits, Y_batch)
        self.log('test_loss', loss_val, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)

        return loss_val

    def predict_step(self, batch, batch_idx):
        X_batch, Y_batch = batch
        preds = self.forward(X_batch.float())
        return preds

    def configure_optimizers(self):
        # optimizer = Adam(self.model.parameters(), lr=learning_rate)
        optimizer = SGD(self.model.parameters(), lr=self.learning_rate, momentum=0.9)
        print("learning_rate = "+str(self.learning_rate))
        scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
        # scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=weight_decay, last_epoch=-1)
        return [optimizer], [scheduler]

    def prepare_data(self):
        # Here you can do wherever you need to have your data ready
        # In this case I'm just downloading the dataset
        #self.df_train = pd.read_csv('/content/drive/MyDrive/thesis/dataset/paddy_doctor/train.csv', encoding='utf-8')
        self.df_train= pd.read_csv('/content/drive/MyDrive/thesis/dataset/paddy_doctor/train/train.csv')
        self.df_valid= pd.read_csv('/content/drive/MyDrive/thesis/dataset/paddy_doctor/val/val.csv')
        #self.df_test = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv', encoding='utf-8')
        pass

    def setup(self, stage=None):
        # Setup function is key when handling dataloaders inside the class since
        # this function is triggered when training or testing the model by passing the
        # right datasets (training, validation and testing)        
        # df_valid = pd.read_csv('data/Treated_Valid_XY.csv', encoding='utf-8')
        
        self.df_train["labels_num"] = self.df_train["label"].apply(lambda x: labels[x])
        self.df_valid["labels_num"] = self.df_valid["label"].apply(lambda x: labels[x])
        #self.df_train = self.df_train.sample(frac=1).reset_index(drop=True)
        #self.df_valid = self.df_train[:1000]
        #self.df_train = self.df_train[1000:]

    def train_dataloader(self):
        # Called when training the model

        y_train = self.df_train.labels_num.values
        class_sample_count = np.array([len(np.where(y_train == t)[0]) for t in np.unique(y_train)])
        weight = 1. / class_sample_count
        samples_weight = np.array([weight[t] for t in y_train])
        samples_weight = torch.from_numpy(samples_weight)

        wt_sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

        self.train_dataset = ImageLabelDataset(image_id=self.df_train["image_id"].values, path=self.df_train["path"].values, y_true=self.df_train["labels_num"].values)
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=False, sampler=wt_sampler)

    def val_dataloader(self):
        # Called when evaluating the model (for each "n" steps or "n" epochs)
        self.val_dataset = ImageLabelDataset(image_id=self.df_valid["image_id"].values, path=self.df_valid["path"].values, y_true=self.df_valid["labels_num"].values, isTrain=1)
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        # Called when testing the model by calling: Trainer.test()
        self.test_dataset = ImageLabelDataset(image_id=self.df_test["image_id"].values, path=self.df_test["path"].values, y_true=self.df_test["labels_num"].values, isTrain=2)
        return DataLoader(self.test_dataset, batch_size=self.batch_size)
    
    def predict_dataloader(self):
        self.test_dataset = ImageLabelDataset(image_id=self.df_test["image_id"].values, path=self.df_test["path"].values, y_true=self.df_test["labels_num"].values, isTrain=2)
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [14]:
MODEL_CKPT_PATH = './'
MODEL_CKPT = 'Effb4-{epoch:02d}-{val_acc:.2f}'

# Hyper parameters
Output_Class = 10
learning_rate = 0.01
weight_decay = 0.000001
batch_size = 10
epochs = 50
#checkpoint_model_name = "models/EFF-Saved-val_acc=0.58.ckpt"

In [15]:
print("Loading Main------")

early_stop_callback = EarlyStopping(
    monitor='val_acc',
    patience=3,
    verbose=False,
    mode='max'
)

checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_CKPT_PATH,
    monitor='val_acc',
    filename=MODEL_CKPT ,
    save_top_k= 1,
    mode='max'
)


classifier = PaddyNeuralNet(learning_rate, batch_size, Output_Class)


print("Module Loaded------")

# pl.seed_everything(42, workers=True)
# ModelPruning("l1_unstructured", amount=0.45)

# tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs/")

trainer = pl.Trainer(
    replace_sampler_ddp=False,
    check_val_every_n_epoch=2,
    max_epochs=epochs, 
    progress_bar_refresh_rate=100,
    log_every_n_steps=100, 
#     logger = tb_logger,
    accelerator="gpu", devices=1,
    callbacks=[checkpoint_callback, early_stop_callback]
)

# trainer.tune(classifier)

# trainer.validate(classifier)

trainer.fit(classifier)

trainer.validate(classifier)

# trainer.test(classifier)

#df_test = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv', encoding='utf-8')
# df_test["labels_num"] = 0

#preds = trainer.predict(classifier)
  

#print(len(preds))
#preds = torch.cat(preds)
#preds = preds.argmax(axis=1)
#print(preds.shape)

#df_test["label"] = preds.detach().cpu().numpy()
#df_test["label"] = df_test["label"].apply(lambda x:rev_labels[x])
# df_final = pd.DataFrame(data=[])
# df_final["original"] = Y_test.detach().cpu().numpy()
# df_final["preds"] = preds.detach().cpu().numpy()

#df_test.to_csv('output.csv', index=False)

print("Ending Main------")


Loading Main------


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=100)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Module Loaded------


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /content exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type             | Params
----------------------------------------------
0 | accuracy | Accuracy         | 0     
1 | model    | EfficientNet     | 17.6 M
2 | loss_fn  | CrossEntropyLoss | 0     
----------------------------------------------
17.3 M    Trainable params
277 K     Non-trainable params
17.6 M    Total params
70.266    Total estimated model params size (MB)


learning_rate = 0.01


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.9599999785423279
        val_loss            0.16034747660160065
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Ending Main------


In [ ]:
# Saving classifier weights
torch.save(classifier.state_dict(), '/content/drive/MyDrive/thesis/dataset/paddy_doctor/paddy_doctor_4_efficientnet-b4/classifier_weights.pth')


In [16]:
torch.save(classifier, '/content/drive/MyDrive/thesis/dataset/paddy_doctor/paddy_doctor_4_efficientnet-b4/classifier_arch_with_weights2.pth')

In [ ]:
# Saving checkpoint
trainer.save_checkpoint("/content/drive/MyDrive/thesis/dataset/paddy_doctor/paddy_doctor_4_efficientnet-b4/epoch_13_val_acc_98.ckpt")

In [ ]:


dummy_input = Variable(torch.randn(1, 3, 380, 380)) 
torch.onnx.export(classifier, dummy_input, "paddy.onnx")

/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_opset9.py:693: UserWarning: ONNX export squeeze with negative axis -1 might cause the onnx model to be incorrect. Negative axis is not supported in ONNX. Axis is converted to 3 based on input shape at export time. Passing an tensor of different rank in execution will be incorrect.
  "Passing an tensor of different rank in execution will be incorrect.")
/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_opset9.py:715: UserWarning: This model contains a squeeze operation on dimension 3. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  "intended to be used with dynamic input shapes, please use opset version 11 to export the model.")
/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_opset9.py:693: UserWarning: ONNX export squeeze with negative axis -1 might cause the onnx model to be incorrect. Negative axis is not supported in ONNX. Axis is converted

In [ ]:
# Load the ONNX file
model1 = onnx.load('paddy.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model1)

In [ ]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

inputs: ['input.1']
outputs: ['1343']
tensor_dict:
{}


In [ ]:
tf_rep.summary()

AttributeError: ignored